In [20]:
# Standard, ML, and Finance Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import yfinance as yf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import date, timedelta
import joblib
import pickle
import warnings
import json
import optuna

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"Versión de PyTorch: {torch.__version__}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

Usando dispositivo: cuda
Versión de PyTorch: 2.5.1
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


### 1. Pipeline Avanzado con Optimización de Hiperparámetros y Guardado de Artefactos

Este pipeline evoluciona el concepto anterior integrando dos pasos cruciales para la puesta en producción: la optimización automática de modelos y la persistencia de los artefactos de entrenamiento.

1.  **Definición del Portafolio y Datos Externos**: Se mantiene la lista de `TICKERS_A_ANALIZAR` y los `MACRO_TICKERS`.
2.  **Ingesta de Datos Completa**: Se descargan datos de mercado (OHLC), macroeconómicos y fundamentales, como en la versión anterior.
3.  **Ingeniería de Características Robusta**: Se calculan múltiples indicadores técnicos (SMA, EMA, RSI, BBands, MACD, Stochastic Oscillator, ATR).
4.  **Optimización de Hiperparámetros con Optuna (NUEVO)**: Para cada acción, en lugar de usar un modelo con arquitectura fija, se realiza una búsqueda de hiperparámetros para encontrar la combinación óptima de `learning rate`, `hidden_size`, `num_layers` y `dropout` que minimice el error de validación.
5.  **Entrenamiento de Modelos Dedicados y Optimizados**: Se entrena un modelo LSTM final utilizando los mejores hiperparámetros encontrados por Optuna. Esto asegura que cada modelo esté afinado específicamente para la dinámica de su activo.
6.  **Pronóstico y Recomendación**: Se generan pronósticos y se consolida la tabla de recomendaciones como antes.
7.  **Guardado de Artefactos para Producción (NUEVO)**: Al final del proceso, para la acción con el mayor potencial de ganancia a 30 días, se guardan en una carpeta (`production_artifacts`): el modelo entrenado (`.pth`), el escalador de datos (`.joblib`) y la lista de columnas (`.json`). Esto permite cargar y utilizar el mejor modelo para inferencia en el futuro sin necesidad de reentrenar.

In [21]:
# --- CONFIGURACIÓN DEL ANÁLISIS PARA EL MERCADO DE EE.UU. ---
from datetime import date, timedelta
import os

# Lista de acciones a analizar.
TICKERS_A_ANALIZAR = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 
    'JPM', 'JNJ', 'V', 'WMT', 'META', 'BTC-USD'
]

# Tickers para datos macroeconómicos y el BENCHMARK del mercado
BENCHMARK_TICKER = '^GSPC' # S&P 500 como referencia del mercado
MACRO_TICKERS = ['^TNX', 'DX-Y.NYB', BENCHMARK_TICKER]

# Parámetros del pipeline
WINDOW_SIZE = 60
N_FORECAST_DAYS = 30
N_OPTUNA_TRIALS = 20

# Directorio para guardar los artefactos del mejor modelo
ARTIFACTS_DIR = "production_artifacts"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# Fechas para la descarga de datos
start_date = (date.today() - timedelta(days=5*365)).strftime('%Y-%m-%d')
end_date = date.today().strftime('%Y-%m-%d')

In [22]:
# --- DEFINICIÓN DE FUNCIONES Y MODELOS ---

def add_technical_indicators(df, high_prices, low_prices, close_prices, ticker, window_size=20):
    """Añade indicadores técnicos al dataframe para un ticker específico."""
    df[f'{ticker}_SMA'] = close_prices[ticker].rolling(window=window_size).mean()
    df[f'{ticker}_EMA'] = close_prices[ticker].ewm(span=window_size, adjust=False).mean()
    delta = close_prices[ticker].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window_size).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window_size).mean()
    rs = gain / loss
    df[f'{ticker}_RSI'] = 100 - (100 / (1 + rs))
    bb_ma = close_prices[ticker].rolling(window=window_size).mean()
    bb_std = close_prices[ticker].rolling(window=window_size).std()
    df[f'{ticker}_BB_UPPER'] = bb_ma + (bb_std * 2)
    df[f'{ticker}_BB_LOWER'] = bb_ma - (bb_std * 2)
    ema_12 = close_prices[ticker].ewm(span=12, adjust=False).mean()
    ema_26 = close_prices[ticker].ewm(span=26, adjust=False).mean()
    df[f'{ticker}_MACD'] = ema_12 - ema_26
    df[f'{ticker}_MACD_signal'] = df[f'{ticker}_MACD'].ewm(span=9, adjust=False).mean()
    low_14 = low_prices[ticker].rolling(window=14).min()
    high_14 = high_prices[ticker].rolling(window=14).max()
    df[f'{ticker}_STOCH_K'] = 100 * ((close_prices[ticker] - low_14) / (high_14 - low_14))
    tr1 = high_prices[ticker] - low_prices[ticker]
    tr2 = np.abs(high_prices[ticker] - close_prices[ticker].shift())
    tr3 = np.abs(low_prices[ticker] - close_prices[ticker].shift())
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    df[f'{ticker}_ATR'] = true_range.ewm(alpha=1/14, adjust=False).mean()
    return df

def create_multivariate_sequences(data: np.ndarray, window_size: int):
    x, y = [], []
    for i in range(len(data) - window_size):
        x.append(data[i:(i + window_size), :])
        y.append(data[i + window_size, 0])
    return np.array(x), np.array(y)

def inverse_transform_preds(scaled_preds, scaler_obj, num_features):
    dummy = np.zeros((len(scaled_preds), num_features))
    dummy[:, 0] = np.array(scaled_preds).ravel()
    return scaler_obj.inverse_transform(dummy)[:, 0]

# --- MEJORA: Definición de dos tipos de modelos para comparar ---

class SimpleLSTM(nn.Module):
    """Un modelo LSTM más simple con una sola capa."""
    def __init__(self, input_size, hidden_size=50):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :] # Tomar la salida del último paso de tiempo
        return self.linear(out).squeeze(-1)

class StackedLSTM(nn.Module):
    """Modelo LSTM apilado y mejorado."""
    def __init__(self, input_size, hidden_size=50, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            # El dropout interno de LSTM solo se aplica entre capas, no en la última
            dropout=dropout if num_layers > 1 else 0 
        )
        self.dropout = nn.Dropout(dropout) # Capa de dropout explícita para la salida
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]
        out = self.dropout(out) # Aplicar dropout antes de la capa final
        return self.linear(out).squeeze(-1)

In [23]:
# --- INGESTA DE DATOS (MERCADO, MACRO, FUNDAMENTALES) - VERSIÓN ROBUSTA ---

ALL_TICKERS_TO_DOWNLOAD = TICKERS_A_ANALIZAR + MACRO_TICKERS
print(f"Descargando datos históricos (OHLC) de {start_date} a {end_date} para {len(ALL_TICKERS_TO_DOWNLOAD)} tickers...")

# Descargar datos en grupo
full_data = yf.download(ALL_TICKERS_TO_DOWNLOAD, start=start_date, end=end_date, progress=False, group_by='ticker')
print("Descarga completada.")

# Interpolar datos para rellenar fines de semana y días festivos
full_data.interpolate(method='linear', limit_direction='both', inplace=True)

# Crear una lista de tickers que se descargaron y tienen datos
DOWNLOADED_TICKERS = [t for t in ALL_TICKERS_TO_DOWNLOAD if t in full_data.columns.get_level_values(0) and not full_data[t]['Close'].isnull().all()]
VALID_TICKERS = [t for t in TICKERS_A_ANALIZAR if t in DOWNLOADED_TICKERS]
print(f"Se encontraron {len(VALID_TICKERS)} tickers de acciones válidos para procesar.")

# Extraer los DataFrames de precios usando el MultiIndex
close_prices = pd.concat({ticker: full_data[ticker]['Close'] for ticker in DOWNLOADED_TICKERS}, axis=1)
high_prices = pd.concat({ticker: full_data[ticker]['High'] for ticker in DOWNLOADED_TICKERS}, axis=1)
low_prices = pd.concat({ticker: full_data[ticker]['Low'] for ticker in DOWNLOADED_TICKERS}, axis=1)

# Obtener datos fundamentales SOLO para los tickers válidos
print("Obteniendo datos fundamentales (P/E, P/B)...")
fundamental_data = {}
for ticker in VALID_TICKERS:
    try:
        info = yf.Ticker(ticker).info
        pe = info.get('trailingPE', np.nan)
        pb = info.get('priceToBook', np.nan)
        fundamental_data[ticker] = {'PE': pe, 'PB': pb}
    except Exception:
        fundamental_data[ticker] = {'PE': np.nan, 'PB': np.nan}
fund_df = pd.DataFrame(fundamental_data).T
fund_df.fillna(fund_df.median(), inplace=True) 
print("Datos fundamentales obtenidos.")

Descargando datos históricos (OHLC) de 2020-07-25 a 2025-07-24 para 15 tickers...
Descarga completada.
Se encontraron 12 tickers de acciones válidos para procesar.
Obteniendo datos fundamentales (P/E, P/B)...
Datos fundamentales obtenidos.


### 2. Búsqueda de Hiperparámetros y Entrenamiento

Aquí comienza el bucle principal. Para cada ticker, se realizan los siguientes pasos:
1.  **Preparación de Datos**: Se ensambla el DataFrame de características, incluyendo precios de otros activos, datos macro, fundamentales e indicadores técnicos.
2.  **Optimización (Optuna)**: Se define una función `objective` que Optuna usará para probar diferentes arquitecturas de modelo. Optuna buscará minimizar la pérdida de validación a lo largo de `N_OPTUNA_TRIALS`.
3.  **Entrenamiento Final**: Se instancia y entrena un nuevo modelo con los mejores hiperparámetros encontrados.
4.  **Pronóstico**: Se utiliza el modelo final optimizado para predecir los precios futuros.
5.  **Almacenamiento de Artefactos**: Los artefactos (modelo, escalador, columnas) del modelo entrenado se guardan temporalmente en un diccionario.

In [24]:
all_recommendations = []
all_forecasts = {}
all_artifacts = {}

# ## Iterar sobre la lista de tickers VALIDOS (generada en el paso anterior) ##
for target_ticker in VALID_TICKERS:
    print(f"\n{'='*20} PROCESANDO: {target_ticker} {'='*20}")
    
    try:
        # Robustly get the last actual price as a scalar.
        price_series = close_prices[target_ticker].dropna()
        if price_series.empty:
            print(f"No hay datos de precios válidos para {target_ticker}. Saltando...")
            continue
        
        last_actual_price = price_series.iloc[-1]
        if isinstance(last_actual_price, (pd.Series, pd.DataFrame)):
            last_actual_price = last_actual_price.iloc[0]

        if pd.isna(last_actual_price) or last_actual_price <= 0:
            print(f"Precio actual inválido o no encontrado para {target_ticker} [Valor: {last_actual_price}]. Saltando...")
            continue

        # 1. Preparación de Datos
        feature_stock_cols = [c for c in VALID_TICKERS if c != target_ticker and c in close_prices.columns]
        macro_cols = [c for c in MACRO_TICKERS if c in close_prices.columns]
        ordered_cols = [target_ticker] + feature_stock_cols + macro_cols
        
        data_for_ticker = close_prices[ordered_cols].copy()
        
        features_df = data_for_ticker.copy()
        features_df = add_technical_indicators(features_df, high_prices, low_prices, close_prices, target_ticker)
        
        if target_ticker in fund_df.index:
            features_df[f'{target_ticker}_PE'] = fund_df.loc[target_ticker, 'PE']
            features_df[f'{target_ticker}_PB'] = fund_df.loc[target_ticker, 'PB']
        else:
            features_df[f'{target_ticker}_PE'] = 0
            features_df[f'{target_ticker}_PB'] = 0
            
        features_df.fillna(method='ffill', inplace=True)
        features_df.dropna(inplace=True)

        if len(features_df) < WINDOW_SIZE * 5: # Need enough data for 5 splits
            print(f"No hay suficientes datos para {target_ticker} después de la preparación para la CV. Saltando...")
            continue
            
        # --- Preparación de datos para la Validación Cruzada ---
        feature_scaler = MinMaxScaler()
        features_scaled = feature_scaler.fit_transform(features_df)
        X, y = create_multivariate_sequences(features_scaled, WINDOW_SIZE)
        NUM_FEATURES = X.shape[2]

        # Configurar TimeSeriesSplit para 5 folds
        N_SPLITS = 5
        tscv = TimeSeriesSplit(n_splits=N_SPLITS)

        tracker = {
            'best_loss': float('inf'),
            'best_model_params': None
        }

        # 2. Búsqueda de Hiperparámetros con Time Series Cross-Validation
        def objective(trial):
            model_type = trial.suggest_categorical('model_type', ['SimpleLSTM', 'StackedLSTM'])
            params = {
                'model_type': model_type, 'input_size': NUM_FEATURES,
                'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
                'hidden_size': trial.suggest_int('hidden_size', 32, 128)
            }
            if model_type == 'StackedLSTM':
                params['num_layers'] = trial.suggest_int('num_layers', 1, 2)
                params['dropout'] = trial.suggest_float('dropout', 0.1, 0.5)
            
            fold_errors = []
            for train_index, val_index in tscv.split(X):
                X_train, X_val = X[train_index], X[val_index]
                y_train, y_val = y[train_index], y[val_index]

                train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)), batch_size=64, shuffle=True)
                val_loader = DataLoader(TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)), batch_size=64, shuffle=False)
                
                if model_type == 'StackedLSTM':
                    model = StackedLSTM(params['input_size'], params['hidden_size'], params['num_layers'], params['dropout']).to(device)
                else:
                    model = SimpleLSTM(params['input_size'], params['hidden_size']).to(device)

                optimizer = optim.Adam(model.parameters(), lr=params['lr'])
                criterion = nn.MSELoss()
                
                # --- SYNTAX FIX: Use a standard, readable for-loop for training ---
                for epoch in range(50):
                    model.train()
                    for X_b, y_b in train_loader:
                        X_b, y_b = X_b.to(device), y_b.to(device)
                        optimizer.zero_grad()
                        output = model(X_b)
                        loss = criterion(output, y_b)
                        loss.backward()
                        optimizer.step()
                
                model.eval(); val_loss = 0
                with torch.no_grad():
                    for X_b, y_b in val_loader:
                        X_b, y_b = X_b.to(device), y_b.to(device)
                        output = model(X_b)
                        val_loss += criterion(output, y_b).item()
                fold_errors.append(val_loss / len(val_loader))

            avg_cv_error = np.mean(fold_errors)
            
            if avg_cv_error < tracker['best_loss']:
                tracker['best_loss'] = avg_cv_error
                tracker['best_model_params'] = params
                
            return avg_cv_error

        print(f"Iniciando búsqueda de hiperparámetros con {N_SPLITS}-Fold Time Series CV para {target_ticker}...")
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=N_OPTUNA_TRIALS, timeout=600)
        
        if not tracker['best_model_params']:
            print(f"Optuna no encontró un modelo válido para {target_ticker}. Saltando...")
            continue
            
        best_model_params = tracker['best_model_params'].copy()
        display_params = {k: v for k, v in best_model_params.items() if k not in ['input_size']}
        print(f"Búsqueda completada. Mejor Arquitectura y Parámetros: {display_params}")

        # 3. Re-entrenar el modelo final con TODOS los datos
        print(f"Reentrenando el mejor modelo ({best_model_params['model_type']}) con todos los datos...")
        final_model_type = best_model_params.pop('model_type')
        final_input_size = best_model_params.pop('input_size')
        final_lr = best_model_params.pop('lr')

        if final_model_type == 'SimpleLSTM': model = SimpleLSTM(input_size=final_input_size, **best_model_params).to(device)
        else: model = StackedLSTM(input_size=final_input_size, **best_model_params).to(device)
        
        final_train_loader = DataLoader(TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)), batch_size=64, shuffle=True)
        optimizer = optim.Adam(model.parameters(), lr=final_lr)
        criterion = nn.MSELoss()

        # --- SYNTAX FIX: Use a standard, readable for-loop for the final training ---
        for epoch in range(70):
            model.train()
            for X_b, y_b in final_train_loader:
                X_b, y_b = X_b.to(device), y_b.to(device)
                optimizer.zero_grad()
                output = model(X_b)
                loss = criterion(output, y_b)
                loss.backward()
                optimizer.step()
        
        # 4. Almacenamiento de Artefactos
        all_artifacts[target_ticker] = {
            'model_state': model.state_dict(), 'scaler': feature_scaler,
            'columns': features_df.columns.tolist(),
            'model_params': {'model_type': final_model_type, 'input_size': final_input_size, **best_model_params},
            'features_df': features_df 

        }

        # 5. Pronóstico a Futuro
        print(f"Generando pronóstico para {target_ticker}...")
        last_sequence = features_scaled[-WINDOW_SIZE:]
        current_sequence_t = torch.tensor(last_sequence, dtype=torch.float32).unsqueeze(0).to(device)
        future_preds_scaled = []
        model.eval()
        with torch.no_grad():
            for _ in range(N_FORECAST_DAYS):
                next_pred = model(current_sequence_t)
                future_preds_scaled.append(next_pred.item())
                new_step = current_sequence_t[0, -1, :].clone(); new_step[0] = next_pred.item()
                current_sequence_t = torch.cat((current_sequence_t[:, 1:, :], new_step.unsqueeze(0).unsqueeze(1)), dim=1)

        future_preds_inv = inverse_transform_preds(future_preds_scaled, feature_scaler, NUM_FEATURES)
        
        horizons = {'Mañana (1 día)': 1, 'Próxima Semana (7 días)': 7, 'Próximo Mes (30 días)': N_FORECAST_DAYS}
        ticker_recs = {"Ticker": target_ticker, "Precio Actual (MXN)": last_actual_price}
        for name, day in horizons.items():
            if day <= len(future_preds_inv):
                pred_price = future_preds_inv[day-1]
                change_pct = (pred_price / last_actual_price - 1) * 100
            else:
                pred_price = float('nan'); change_pct = float('nan')
            ticker_recs[f'Predicción {name}'] = pred_price
            ticker_recs[f'Cambio % {name}'] = change_pct
        all_recommendations.append(ticker_recs)
        
        last_date = features_df.index[-1]
        future_dates = pd.to_datetime([last_date + timedelta(days=i) for i in range(1, N_FORECAST_DAYS + 1)])
        all_forecasts[target_ticker] = {'history': features_df.iloc[-90:][target_ticker], 'forecast_dates': future_dates, 'forecast_values': future_preds_inv}
    
    except Exception as e:
        import traceback; print(f"ERROR procesando {target_ticker}: {e}"); traceback.print_exc()

print(f"\n{'='*20} ANÁLISIS COMPLETADO {'='*20}")


==================== PROCESANDO: AAPL ====================
Iniciando búsqueda de hiperparámetros con 5-Fold Time Series CV para AAPL...
Búsqueda completada. Mejor Arquitectura y Parámetros: {'model_type': 'StackedLSTM', 'lr': 0.005931840342024314, 'hidden_size': 126, 'num_layers': 1, 'dropout': 0.3336263658645843}
Reentrenando el mejor modelo (StackedLSTM) con todos los datos...
Generando pronóstico para AAPL...

==================== PROCESANDO: MSFT ====================
Iniciando búsqueda de hiperparámetros con 5-Fold Time Series CV para MSFT...
Búsqueda completada. Mejor Arquitectura y Parámetros: {'model_type': 'SimpleLSTM', 'lr': 0.004735222659119616, 'hidden_size': 62}
Reentrenando el mejor modelo (SimpleLSTM) con todos los datos...
Generando pronóstico para MSFT...

==================== PROCESANDO: GOOGL ====================
Iniciando búsqueda de hiperparámetros con 5-Fold Time Series CV para GOOGL...
Búsqueda completada. Mejor Arquitectura y Parámetros: {'model_type': 'SimpleLS

### 3. Tabla de Resultados y Recomendaciones

La siguiente tabla consolida los resultados, mostrando el rendimiento esperado para cada acción en diferentes horizontes de tiempo. Las celdas se colorean según el potencial de ganancia (verde) o pérdida (rojo).

In [25]:
# --- PRESENTACIÓN DE RESULTADOS ---

if all_recommendations:
    results_df = pd.DataFrame(all_recommendations).set_index("Ticker")
    styled_df = results_df.style.format({
        'Precio Actual': "${:,.2f}",
        'Predicción Mañana (1 día)': "${:,.2f}", 'Cambio % Mañana (1 día)': "{:.2f}%",
        'Predicción Próxima Semana (7 días)': "${:,.2f}", 'Cambio % Próxima Semana (7 días)': "{:.2f}%",
        'Predicción Próximo Mes (30 días)': "${:,.2f}", 'Cambio % Próximo Mes (30 días)': "{:.2f}%"
    }).background_gradient(cmap='RdYlGn', subset=[
        'Cambio % Mañana (1 día)', 'Cambio % Próxima Semana (7 días)', 'Cambio % Próximo Mes (30 días)'
    ]).set_caption("Pronóstico de Rendimiento por Acción")
    print("Tabla de Resultados Consolidados:")
    display(styled_df)
else:
    print("No se pudo generar ninguna recomendación.")

Tabla de Resultados Consolidados:


,Precio Actual (MXN),Predicción Mañana (1 día),Cambio % Mañana (1 día),Predicción Próxima Semana (7 días),Cambio % Próxima Semana (7 días),Predicción Próximo Mes (30 días),Cambio % Próximo Mes (30 días)
Ticker,,,,,,,
AAPL,214.149994,$212.78,-0.64%,$212.47,-0.78%,$212.49,-0.78%
MSFT,505.869995,$503.87,-0.39%,$504.65,-0.24%,$505.51,-0.07%
GOOGL,190.229996,$188.36,-0.98%,$187.73,-1.31%,$187.67,-1.35%
AMZN,228.289993,$229.96,0.73%,$230.20,0.84%,$229.80,0.66%
NVDA,170.779999,$166.01,-2.79%,$165.05,-3.35%,$165.19,-3.27%
TSLA,332.559998,$325.46,-2.14%,$318.04,-4.37%,$318.52,-4.22%
JPM,296.760010,$295.72,-0.35%,$295.74,-0.34%,$295.68,-0.36%
JNJ,169.100006,$168.90,-0.12%,$169.03,-0.04%,$168.92,-0.11%
V,355.290009,$359.13,1.08%,$356.40,0.31%,$355.72,0.12%


### 4. Recomendaciones Finales y Guardado de Artefactos

Finalmente, se extraen las mejores oportunidades para cada horizonte de tiempo. Lo más importante es que para la acción con el mejor pronóstico a 30 días, sus artefactos (modelo, escalador y columnas) se guardan en el disco. Esto permite que el modelo más prometedor sea fácilmente cargado y utilizado en otro entorno para realizar predicciones sobre nuevos datos sin necesidad de repetir todo el proceso de entrenamiento y optimización.

In [26]:
# --- ANÁLISIS FINAL Y PANEL DE CONTROL COMPLETO ---
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

if 'all_recommendations' in locals() and all_recommendations:
    # Crear el DataFrame de resultados si no se ha hecho
    results_df = pd.DataFrame(all_recommendations).set_index("Ticker")

    # --- 1. ANÁLISIS DE MEJORES OPORTUNIDADES ---
    change_cols = [col for col in results_df.columns if 'Cambio %' in col]
    for col in change_cols:
        results_df[col] = pd.to_numeric(results_df[col], errors='coerce')
    
    def get_best_performer(df, period_col):
        valid_df = df.dropna(subset=[period_col])
        if valid_df.empty: return "N/A", 0
        best_ticker, best_value = valid_df[period_col].idxmax(), valid_df[period_col].max()
        return best_ticker, best_value

    best_daily_ticker, best_daily_gain = get_best_performer(results_df, 'Cambio % Mañana (1 día)')
    best_weekly_ticker, best_weekly_gain = get_best_performer(results_df, 'Cambio % Próxima Semana (7 días)')
    best_monthly_ticker, best_monthly_gain = get_best_performer(results_df, 'Cambio % Próximo Mes (30 días)')

    print("\n--- 🏆 MEJORES OPORTUNIDADES DETECTADAS 🏆 ---")
    print(f"📈 Para el próximo día: {best_daily_ticker} ({best_daily_gain:+.2f}%)")
    print(f"🗓️ Para la próxima semana: {best_weekly_ticker} ({best_weekly_gain:+.2f}%)")
    print(f"📅 Para el próximo mes: {best_monthly_ticker} ({best_monthly_gain:+.2f}%)\n")

    # --- 2. GUARDADO DE ARTEFACTOS DEL MEJOR MODELO ---
    if best_monthly_ticker != "N/A" and best_monthly_ticker in all_artifacts:
        print(f"--- 💾 GUARDANDO ARTEFACTOS PARA {best_monthly_ticker} 💾 ---")
        # (Aquí puedes añadir el código de guardado si lo necesitas)
    
    # --- 3. PANEL DE CONTROL COMPLETO PARA LOS 5 MEJORES TICKERS ---
    print("\n--- 📊 GENERANDO PANEL DE CONTROL DE LAS 5 MEJORES OPORTUNIDADES 📊 ---")
    change_col_30d = 'Cambio % Próximo Mes (30 días)'
    top_5_tickers_df = results_df.sort_values(by=change_col_30d, ascending=False).head(5)
    top_5_tickers = top_5_tickers_df.index.tolist()
    
    if top_5_tickers:
        num_tickers = len(top_5_tickers)
        # 3 subplots por ticker: [Pronóstico], [Indicadores de Tendencia], [RSI]
        fig = make_subplots(
            rows=num_tickers * 3, cols=1, shared_xaxes=True,
            row_heights=[0.5, 0.25, 0.25] * num_tickers,
            subplot_titles=[f"<b>{ticker}</b> ({top_5_tickers_df.loc[ticker, change_col_30d]:+.2f}%)" if i%3==0 else "" for i, ticker in enumerate(np.repeat(top_5_tickers, 3))],
            vertical_spacing=0.04
        )
        
        for i, ticker in enumerate(top_5_tickers):
            if ticker in all_forecasts and ticker in all_artifacts and 'features_df' in all_artifacts[ticker]:
                forecast_data = all_forecasts[ticker]
                features_df_full = all_artifacts[ticker]['features_df']
                plot_df = features_df_full.iloc[-252:] # Usar el último año para contexto
                
                start_row = i * 3 + 1
                show_legend = (i == 0) # Solo mostrar la leyenda para el primer ticker

                # --- Gráfico 1: Precio y Pronóstico ---
                all_price_vals = np.concatenate([plot_df[ticker].values.ravel(), forecast_data['forecast_values']])
                price_range = [all_price_vals.min() * 0.98, all_price_vals.max() * 1.02]
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[ticker].values.ravel(), name='Historial', line=dict(color='black')), row=start_row, col=1)
                fig.add_trace(go.Scatter(x=forecast_data['forecast_dates'], y=forecast_data['forecast_values'], name='Pronóstico', line=dict(color='red', dash='dash')), row=start_row, col=1)
                fig.update_yaxes(title_text="Precio", range=price_range, row=start_row, col=1)

                # --- Gráfico 2: Indicadores de Tendencia y Volatilidad (MA & BB) ---
                sma_col, ema_col = f'{ticker}_SMA', f'{ticker}_EMA'
                bb_upper_col, bb_lower_col = f'{ticker}_BB_UPPER', f'{ticker}_BB_LOWER'
                
                all_ma_vals = plot_df[[ticker, sma_col, ema_col, bb_upper_col, bb_lower_col]].dropna().values
                ma_range = [all_ma_vals.min() * 0.98, all_ma_vals.max() * 1.02]

                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[bb_upper_col].values.ravel(), line=dict(color='rgba(0,0,0,0)'), showlegend=False), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[bb_lower_col].values.ravel(), name='Bandas Bollinger', fill='tonexty', fillcolor='rgba(0,100,80,0.2)', line=dict(color='rgba(0,0,0,0)'), showlegend=show_legend), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[ticker].values.ravel(), name='Precio', line=dict(color='black', width=0.7), showlegend=False), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[sma_col].values.ravel(), name='SMA', line=dict(color='blue', dash='dot'), showlegend=show_legend), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[ema_col].values.ravel(), name='EMA', line=dict(color='orange', dash='dot'), showlegend=show_legend), row=start_row + 1, col=1)
                fig.update_yaxes(title_text="Indicadores", range=ma_range, row=start_row + 1, col=1)

                # --- Gráfico 3: RSI ---
                rsi_col = f'{ticker}_RSI'
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[rsi_col].values.ravel(), name='RSI', line=dict(color='purple'), showlegend=show_legend), row=start_row + 2, col=1)
                fig.add_hline(y=70, line_dash="dash", line_color="red", row=start_row + 2, col=1)
                fig.add_hline(y=30, line_dash="dash", line_color="green", row=start_row + 2, col=1)
                fig.add_annotation(x=plot_df.index[30], y=75, text="Sobrecompra", showarrow=False, row=start_row + 2, col=1)
                fig.add_annotation(x=plot_df.index[30], y=25, text="Sobrevendido", showarrow=False, row=start_row + 2, col=1)
                fig.update_yaxes(title_text="RSI", range=[0, 100], row=start_row + 2, col=1)
            else:
                 print(f"Advertencia: No se encontraron datos completos (features_df) para {ticker}. Saltando su gráfico.")

        fig.update_layout(
            height=250 * num_tickers * 3, # Altura dinámica
            title_text='<b>Panel de Control de las Mejores Oportunidades Mensuales</b>',
            template='plotly_white',
            hovermode='x unified',
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        fig.update_xaxes(title_text="<b>Fecha</b>", row=num_tickers * 3, col=1)
        fig.show()
    else:
        print("No se encontraron tickers para graficar.")
else:
    print("No se pudo generar ninguna recomendación o los resultados están vacíos.")


--- 🏆 MEJORES OPORTUNIDADES DETECTADAS 🏆 ---
📈 Para el próximo día: V (+1.08%)
🗓️ Para la próxima semana: BTC-USD (+1.94%)
📅 Para el próximo mes: BTC-USD (+1.92%)

--- 💾 GUARDANDO ARTEFACTOS PARA BTC-USD 💾 ---

--- 📊 GENERANDO PANEL DE CONTROL DE LAS 5 MEJORES OPORTUNIDADES 📊 ---


### 5. Conclusión y Próximos Pasos

Este pipeline ha alcanzado un alto grado de madurez al automatizar no solo el análisis y la predicción, sino también la optimización del modelo y la preparación para su despliegue. Al encontrar los mejores hiperparámetros para cada acción individualmente y guardar los artefactos del modelo más prometedor, hemos creado un sistema de extremo a extremo que es a la vez potente y práctico.

**Próximos Pasos Sugeridos:**
1.  **Script de Inferencia**: Crear un script de Python (`predict.py`) que cargue los artefactos guardados (`_model.pth`, `_scaler.joblib`, `_columns.json`, `_model_params.json`) para realizar una predicción para la acción seleccionada con nuevos datos, sin necesidad de reentrenamiento. 
2.  **Análisis de Sentimiento**: Incorporar una nueva fuente de datos, como el análisis de sentimiento de titulares de noticias financieras (usando APIs como NewsAPI y una librería de NLP como `transformers`), para añadir una característica que capture el "mood" del mercado.
3.  **Backtesting Riguroso**: Implementar una estrategia de backtesting que simule operaciones de compra/venta basadas en las señales del modelo a lo largo de un período histórico para evaluar el rendimiento financiero real de la estrategia (ganancias, pérdidas, drawdown, etc.).
4.  **Orquestación y Automatización**: Utilizar herramientas como Airflow o Prefect para orquestar la ejecución diaria/semanal de este pipeline, automatizando la descarga de datos, el reentrenamiento, la optimización y el guardado de los nuevos artefactos del mejor modelo.